# Bubble Point (VLE)

The isothermal-isobaric two-phase flash is the base for the calculation Vapor-Liquid Equilibria. This calculation is based on the solution of the Rachford-Rice mass balance. 

$$ FO = \sum_{i=1}^c \left( x_i^\beta - x_i^\alpha \right) = \sum_{i=1}^c \frac{z_i (K_i-1)}{1+\psi (K_i-1)} $$


Where,  $K_i = x_i^\beta/x_i^\alpha =\hat{\phi}_i^\alpha /\hat{\phi}_i^\beta  $ represents the equilibrium constant  and $\psi$ the fraction of the phase $\beta$. For bubble and dew points calculations the phase fraction $\psi$ is known beforehand and set to 0 for bubble points (differential size bubble) and to 1 for dew point (differential size liquid drop).

The Rachford-Rice mass balance reduces to the following equations:

### Bubble

$$ FO = \sum_{i=1}^c x_i (K_i-1) = \sum_{i=1}^c y_i -1 = 0 $$

The solution of this calculations includes using accelerated successive substitution (ASS) to update the phase compositions in an inner loop and the quasi-Newton method is used to update pressure or temperature in an outer loop. If slow convergence is detected, the algorithm attempts to solve the following system of equations using equilibrium constants, $K$, as iteration variables. This is done using SciPy's optimization routines.

$$ f_i = \ln K_i + \ln \hat{\phi}_i^v(\underline{y}, T, P) -\ln \hat{\phi}_i^l(\underline{x}, T, P) \quad i = 1,...,c $$
$$ f_{c+1} = \sum_{i=1}^c (y_i-x_i) $$

**note:** these calculations does not check for the stability of the phases.


In this notebook, examples of calculation of bubble point properties using Peng-Robinson equation of state are shown. The mixing rule applied is Modified Huron Vidal combined with Modified UNIFAC.

In [1]:
import numpy as np
from phasepy import mixture, component, preos
from phasepy.equilibrium import bubblePy, bubbleTy

---
# Binary mixture example

This calculation will be exemplified for the mixture of ethanol and water.

In [2]:
water = component(name='water', Tc=647.13, Pc=220.55, 
                  w=0.344861, GC={'H2O':1})
ethanol = component(name='ethanol', Tc=514.0, Pc=61.37,
                    w=0.643558, GC={'CH3':1, 'CH2':1, 'OH(P)':1})
mix = mixture(ethanol, water)
# or
mix = ethanol + water
mix.unifac() 
eos = preos(mix, 'mhv_unifac')

-----
#### Bubble point algorithm x, T -> y, P

In [3]:
x = np.array([0.5, 0.5])
T = 350.0
y0 = np.array([0.5, 0.5])
P0 = 1.0
bubblePy(y0, P0, x, T, eos, full_output=True)

      T: 350.0
      P: 0.8969829229313796
  error: 1.1465273175303992e-12
   iter: 5
      X: array([0.5, 0.5])
     v1: 44.1406748020394
 state1: 'Liquid'
      Y: array([0.68233401, 0.31766599])
     v2: 31901.550136613874
 state2: 'Vapor'

-----
#### Bubble point algorithm x, P -> y, T

In [4]:
x = np.array([0.6, 0.4])
P = 3.0
y0 = np.array([0.5, 0.5])
T0 = 320.0
bubbleTy(y0, T0, x, P, eos, full_output=True)

      T: 382.84576305426106
      P: 3.0
  error: 2.3354718162123485e-10
   iter: 5
      X: array([0.6, 0.4])
     v1: 51.11127949434453
 state1: 'Liquid'
      Y: array([0.7027532, 0.2972468])
     v2: 10143.393200684473
 state2: 'Vapor'

# Ternary mixture example

In [5]:
mtbe = component(name='mtbe', Tc=497.1, Pc=34.3, Zc=0.273, Vc=329.0, w=0.266059,
                 Ant=[9.16238246, 2541.97883529, -50.40534341], 
                 GC={'CH3':3, 'CH3O':1, 'C':1})

ethanol = component(name='ethanol', Tc=514.0, Pc=61.37, Zc=0.241, Vc=168.0, w=0.643558,
                    Ant=[11.61809279, 3423.0259436, -56.48094263],
                    GC={'CH3':1, 'CH2':1,'OH(P)':1})

butanol = component(name='n-Butanol', Tc=563.0, Pc=44.14, Zc=0.258, Vc=274.0, w=0.589462,
                    Ant=[9.74673479, 2668.52570016, -116.66189545],
                    GC={'CH3':1, 'CH2':3, 'OH(P)':1})

mix = mixture(mtbe, ethanol)
mix.add_component(butanol)
# or 
mix = mtbe + ethanol + butanol

mix.unifac()
eos = preos(mix, 'mhv_unifac')

-----
#### Bubble point algorithm x, T -> y, P

In [6]:
x = np.array([0.2, 0.5, 0.3])
T = 350.0
y0 = np.array([0.2, 0.5, 0.3])
P0 = 1
bubblePy(y0, P0, x, T, eos, full_output=True)

      T: 350.0
      P: 1.279357776059529
  error: 3.520168601056639e-10
   iter: 5
      X: array([0.2, 0.5, 0.3])
     v1: 87.34523143118855
 state1: 'Liquid'
      Y: array([0.5295944 , 0.41590612, 0.05449948])
     v2: 21941.4177971427
 state2: 'Vapor'

-----
#### Bubble point algorithm x, P -> y, T

In [7]:
x = np.array([0.2, 0.5, 0.3])
P = 2.0
y0 = np.array([0.2, 0.5, 0.3])
T0 = 320.0
bubbleTy(y0, T0, x, P, eos, full_output=True)

      T: 364.21636396579174
      P: 2.0
  error: 1.6830981053315957e-13
   iter: 5
      X: array([0.2, 0.5, 0.3])
     v1: 89.37008110948325
 state1: 'Liquid'
      Y: array([0.48385218, 0.45234195, 0.06380586])
     v2: 14396.135655179278
 state2: 'Vapor'

For further information please also check [official documentation](https://phasepy.readthedocs.io/), or just try:

```function?```